In [19]:
import numpy as np
import matplotlib.pyplot as plt

# 데이터 준비
data = np.array([10345.95344093, 10254.57894458, 10198.42042604, 10160.22832452, 10117.95232831, 
                 10096.21754303, 10068.98950732, 10048.45509888, 10029.97294258, 10012.6500211, 
                 9992.32002722, 9986.8216521, 9967.30002051, 9953.65734963, 9944.32238966, 
                 9924.22764642, 9918.56652301, 9906.78991636, 9896.77347545, 9881.72663955, 
                 9870.3843831, 9866.06461084, 9857.33460291, 9847.63331086, 9839.89791331, 
                 9829.26714378, 9819.53021078, 9803.83869316, 9802.0457347, 9792.98681928, 
                 9785.25389664, 9770.42734065, 9762.52207987, 9747.74130149, 9745.77012212, 
                 9734.95618568, 9731.59571095, 9728.38748027, 9713.43131064, 9706.91520543, 
                 9698.41219488, 9695.5943976, 9685.78235559, 9677.83567665, 9678.25361304])

# 데이터 정규화
data_normalized = (data - np.min(data)) / (np.max(data) - np.min(data))

# 입력 데이터 생성
X = np.arange(len(data)).reshape(-1, 1) / len(data)
Y = data_normalized.reshape(-1, 1)



def search_golden_section(fun, dfun, x, s, args=(), delta=1.0e-2, tol=1e-15):
    """
    https://en.wikipedia.org/wiki/Golden-section_search and [arora]
    
    fun   : Original objective function
    dfun  : Objective function gradient which is not used
    x     : Start point
    s     : 1D search directin
    args  : Tuple extra arguments passed to the objective function
    delta : Init. guess interval determining initial interval of uncertainty
    tol   : stop criterion
    """
    gr = (np.sqrt(5) + 1) / 2
        
    AL = 0.
    FL = f_alpha(AL, fun, x, s, args)
    AA = delta
    FA = f_alpha(AA, fun, x, s, args)
    while  FL < FA :
        delta = 0.1*delta
        AA = delta
        FA = f_alpha(AA, fun, x, s, args)
    
    j = 1
    AU = AA + delta * (gr**j)
    FU = f_alpha(AU, fun, x, s, args)
    while FA > FU :
        AL = AA
        AA = AU
        FL = FA
        FA = FU
        
        j += 1
        AU = AA + delta * (gr**j)
        FU = f_alpha(AU, fun, x, s, args)

    AB = AL + (AU - AL) / gr
    FB = f_alpha(AB, fun, x, s, args)
    
    while abs(AA - AB) > tol:
        if f_alpha(AA, fun, x, s, args) < f_alpha(AB, fun, x, s, args):
            AU = AB
        else:
            AL = AA

        # we recompute both c and d here to avoid loss of precision 
        # which may lead to incorrect results or infinite loop
        AA = AU - (AU - AL) / gr
        AB = AL + (AU - AL) / gr

    return ( (AU + AL) / 2, )


def compute_conjugate_gradient(f, df, x, args=(), eps=1.0e-7, max_iter=2500, verbose=False, callback=None):
    """
    f       : 최적화 할 함수
    df      : 최적화할 함수의 도함수
    x       : 초기값
    args    : f(x, *args)로 f를 호출하기 위한 추가 매개변수
    eps     : 정기 기준
    max_iter: 최대 반복수
    verbose : 실행 정보를 텍스트로 출력 
    callback: x에 대한 기록을 위한 콜백함수
    """

    if verbose:
        print("#####START OPTIMIZATION#####")
        print("INIT POINT : {}, dtype : {}".format(x, x.dtype))

    for k in range(max_iter):
        c = df(x, *args)
        if np.linalg.norm(c) < eps :
            if verbose:
                print("Stop criterion break Iter: {:5d}, x: {}".format(k, x))
                print("\n")
            break

        if k == 0 :
            d = -c
        else:
            beta = (np.linalg.norm(c) / np.linalg.norm(c_old))**2
            d = -c + beta*d
        
        alpha = search_golden_section(f, df, x, d, args=args)[0]
        x = x + alpha * d
        c_old = c.copy()    

        if callback :
            callback(x)    

    else:
        print("Stop max iter:{:5d} x:{}".format(k, x)) 

    return x


# 신경망 모델 정의
class NeuralNetwork:
    def __init__(self, input_size, hidden_sizes, output_size):
        self.layers = len(hidden_sizes) + 1
        self.W = [np.random.randn(input_size, hidden_sizes[0]) * 0.01]
        self.b = [np.zeros((1, hidden_sizes[0]))]
        
        for i in range(1, len(hidden_sizes)):
            self.W.append(np.random.randn(hidden_sizes[i-1], hidden_sizes[i]) * 0.01)
            self.b.append(np.zeros((1, hidden_sizes[i])))
        
        self.W.append(np.random.randn(hidden_sizes[-1], output_size) * 0.01)
        self.b.append(np.zeros((1, output_size)))

    def forward(self, X):
        self.a = [X]
        for i in range(self.layers):
            z = np.dot(self.a[i], self.W[i]) + self.b[i]
            a = np.tanh(z)
            self.a.append(a)
        return self.a[-1]

    def get_params(self):
        return np.concatenate([w.ravel() for w in self.W] + [b.ravel() for b in self.b])

    def set_params(self, params):
        start = 0
        for i in range(self.layers):
            end = start + self.W[i].size
            self.W[i] = params[start:end].reshape(self.W[i].shape)
            start = end
            end = start + self.b[i].size
            self.b[i] = params[start:end].reshape(self.b[i].shape)
            start = end

# 목적 함수 및 그래디언트 정의
def objective(params, nn, X, Y):
    nn.set_params(params)
    predictions = nn.forward(X)
    return np.mean((predictions - Y) ** 2)

def gradient(params, nn, X, Y):
    nn.set_params(params)
    m = X.shape[0]
    predictions = nn.forward(X)
    
    grad = []
    delta = predictions - Y
    for i in range(nn.layers - 1, -1, -1):
        dW = np.dot(nn.a[i].T, delta) / m
        db = np.sum(delta, axis=0, keepdims=True) / m
        grad = [dW.ravel(), db.ravel()] + grad
        
        if i > 0:
            delta = np.dot(delta, nn.W[i].T) * (1 - nn.a[i]**2)
    
    return np.concatenate(grad)

# Helper function for search_golden_section
def f_alpha(alpha, fun, x, s, args):
    return fun(x + alpha * s, *args)

# search_golden_section 및 compute_conjugate_gradient 함수는 이미 정의되어 있다고 가정합니다.

# 모델 학습
nn = NeuralNetwork(input_size=1, hidden_sizes=[32, 32], output_size=1)
initial_params = nn.get_params()

optimized_params = compute_conjugate_gradient(
    f=objective,
    df=gradient,
    x=initial_params,
    args=(nn, X, Y),
    eps=1e-6,
    max_iter=1000,
    verbose=True
)

nn.set_params(optimized_params)

# 예측
predictions_normalized = nn.forward(X)
predictions = predictions_normalized * (np.max(data) - np.min(data)) + np.min(data)

# 결과 시각화
plt.figure(figsize=(10, 5))
plt.plot(data, label='Actual')
plt.plot(predictions, label='Predicted', linestyle='--')
plt.title('Time Series Prediction with Conjugate Gradient Optimization')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.show()

# 성능 평가
mse = np.mean((data - predictions.flatten()) ** 2)
print(f'Mean Squared Error: {mse}')

NameError: name 'compute_conjugate_gradient' is not defined